In [1]:
import rasterio
tif_path = r"E:\TreeExtraction\Data_v1\Train_Image.tif"

with rasterio.open(tif_path) as src:
    print("File:", tif_path)
    print("Driver:", src.driver)  
    print("CRS:", src.crs)        
    print("Width (pixels):", src.width)
    print("Height (pixels):", src.height)
    print("Pixel Size:", src.res) 
    print("Number of Bands:", src.count)
    print("Bounds:", src.bounds)  
    print("Data Type:", src.dtypes)
    print("Transform:", src.transform)
    
    for i in range(1, src.count + 1):
        print(f" Band {i}: dtype={src.dtypes[i-1]}, min={src.read(i).min()}, max={src.read(i).max()}")


File: E:\TreeExtraction\Data_v1\Train_Image.tif
Driver: GTiff
CRS: ESRI:54004
Width (pixels): 5296
Height (pixels): 3009
Pixel Size: (0.08556790030251885, 0.08556789631100144)
Number of Bands: 3
Bounds: BoundingBox(left=-19519353.0406, bottom=-2388477.5968, right=-19518899.873, top=-2388220.123)
Data Type: ('uint8', 'uint8', 'uint8')
Transform: | 0.09, 0.00,-19519353.04|
| 0.00,-0.09,-2388220.12|
| 0.00, 0.00, 1.00|
 Band 1: dtype=uint8, min=0, max=255
 Band 2: dtype=uint8, min=0, max=255
 Band 3: dtype=uint8, min=0, max=255


In [1]:
import geopandas as gpd

shp_path = r"E:\TreeExtraction\Data_v1\Traning_Sample_Buffer.shp"
gdf = gpd.read_file(shp_path)

print("File:", shp_path)
print("CRS:", gdf.crs)                    
print("Number of Features:", len(gdf))    
print("Geometry Type:", gdf.geom_type.unique())  
print("Bounds:", gdf.total_bounds)     

print("\nAttribute Fields:")
print(list(gdf.columns))

print("\nFirst row:")
print(gdf.head(1))


File: E:\TreeExtraction\Data_v1\Traning_Sample_Buffer.shp
CRS: PROJCS["World_Mercator",GEOGCS["WGS 84",DATUM["WGS_1984",SPHEROID["WGS 84",6378137,298.257223563,AUTHORITY["EPSG","7030"]],AUTHORITY["EPSG","6326"]],PRIMEM["Greenwich",0,AUTHORITY["EPSG","8901"]],UNIT["degree",0.0174532925199433,AUTHORITY["EPSG","9122"]],AUTHORITY["EPSG","4326"]],PROJECTION["Mercator_2SP"],PARAMETER["standard_parallel_1",0],PARAMETER["central_meridian",0],PARAMETER["false_easting",0],PARAMETER["false_northing",0],UNIT["metre",1,AUTHORITY["EPSG","9001"]],AXIS["Easting",EAST],AXIS["Northing",NORTH],AUTHORITY["ESRI","54004"]]
Number of Features: 645
Geometry Type: ['Polygon']
Bounds: [-19519355.04801589  -2388479.83524714 -19518920.31025364
  -2388217.74357111]

Attribute Fields:
['osm_id', 'name', 'barrier', 'highway', 'ref', 'address', 'is_in', 'place', 'man_made', 'other_tags', 'geometry']

First row:
       osm_id  name barrier highway   ref address is_in place man_made  \
0  5271769476  None    None    No

In [7]:
import matplotlib.pyplot as plt
import rasterio
from rasterio.plot import show
import geopandas as gpd

tif_path = r"E:\TreeExtraction\Data_v1\Train_Image.tif"
shp_path = r"E:\TreeExtraction\Data_v1\Traning_Sample_Buffer.shp"

src = rasterio.open(tif_path)
gdf = gpd.read_file(shp_path)

fig, ax = plt.subplots(figsize=(12, 12))
show((src, [1, 2, 3]), ax=ax)

gdf.plot(ax=ax, marker="o", color="red", markersize=3, alpha=0.6, label="Tree Points")
plt.legend()
plt.title("RGB Raster + Trees Overlay", fontsize=14)
plt.show()


C:\Users\subha\AppData\Local\Temp\ipykernel_19572\4185628133.py:16: UserWarning: Legend does not support handles for PatchCollection instances.
See: https://matplotlib.org/stable/tutorials/intermediate/legend_guide.html#implementing-a-custom-legend-handler
  plt.legend()
C:\Users\subha\AppData\Local\Temp\ipykernel_19572\4185628133.py:16: UserWarning: No artists with labels found to put in legend.  Note that artists whose label start with an underscore are ignored when legend() is called with no argument.
  plt.legend()


In [3]:
import os
import rasterio
from rasterio.windows import Window
from shapely.geometry import box
import geopandas as gpd
from tqdm import tqdm

big_tif   = r"E:\TreeExtraction\Data_v1\Train_Image.tif"
shp_path  = r"E:\TreeExtraction\Data_v1\Traning_Sample_Buffer.shp"
out_img   = r"E:\TreeExtraction\Data_v1\patches\images"
out_lbl   = r"E:\TreeExtraction\Data_v1\patches\labels"

os.makedirs(out_img, exist_ok=True)
os.makedirs(out_lbl, exist_ok=True)

patch_size = 256
overlap    = 128

gdf = gpd.read_file(shp_path)
with rasterio.open(big_tif) as src:
    gdf = gdf.to_crs(src.crs)

    W, H = src.width, src.height
    def poly_to_yolo(polygon, img_w, img_h, transform):
        minx, miny, maxx, maxy = polygon.bounds
        col_min, row_max = ~transform * (minx, miny)
        col_max, row_min = ~transform * (maxx, maxy)

        col_min, col_max = max(0, col_min), min(img_w, col_max)
        row_min, row_max = max(0, row_min), min(img_h, row_max)

        if col_max <= col_min or row_max <= row_min:
            return None

        x_center = ((col_min + col_max) / 2) / img_w
        y_center = ((row_min + row_max) / 2) / img_h
        w = (col_max - col_min) / img_w
        h = (row_max - row_min) / img_h
        return [0, x_center, y_center, w, h]

    step = patch_size - overlap
    counter = 0        # total patches
    label_counter = 0  # patches with labels

    for i in tqdm(range(0, W, step)):
        for j in range(0, H, step):
            if i + patch_size > W or j + patch_size > H:
                continue

            window = Window(i, j, patch_size, patch_size)
            transform = src.window_transform(window)
            patch_bounds = box(*rasterio.windows.bounds(window, transform=src.transform))

            sub_gdf = gdf[gdf.intersects(patch_bounds)]
            if len(sub_gdf) == 0:
                continue

            patch_img = src.read(window=window)

            patch_name = f"image_{counter:03d}.tif"
            img_path = os.path.join(out_img, patch_name)
            profile = src.profile
            profile.update({
                "height": patch_size,
                "width": patch_size,
                "transform": transform
            })
            with rasterio.open(img_path, "w", **profile) as dst:
                dst.write(patch_img)

            yolo_lines = []
            for geom in sub_gdf.geometry:
                yolo_box = poly_to_yolo(geom, patch_size, patch_size, transform)
                if yolo_box:
                    yolo_lines.append(" ".join(map(str, yolo_box)))

            if len(yolo_lines) > 0:
                label_path = os.path.join(out_lbl, patch_name.replace(".tif", ".txt"))
                with open(label_path, "w") as f:
                    f.write("\n".join(yolo_lines))
                label_counter += 1

            counter += 1

print(f"✅ Total patches created: {counter}")
print(f"✅ Patches with labels:   {label_counter}")

100%|██████████████████████████████████████████████████████████████████████████████████| 42/42 [01:04<00:00,  1.54s/it]

✅ Total patches created: 772
✅ Patches with labels:   772


In [1]:
import os
from glob import glob

patch_dir = r"E:\TreeExtraction\Data_v1\patches\images"
label_dir = r"E:\TreeExtraction\Data_v1\patches\labels"

patches = sorted(glob(os.path.join(patch_dir, "*.tif")))

print(f"🔎 Found {len(patches)} patch images")
if len(patches) < 5:
    print("❌ Not enough patches, stopping...")
else:
    print("✅ Enough patches found, proceeding...")

labels = sorted(glob(os.path.join(label_dir, "*.txt")))
print(f"🔎 Found {len(labels)} label files")


🔎 Found 772 patch images
✅ Enough patches found, proceeding...
🔎 Found 772 label files


In [3]:
import os
import random
import shutil

IMAGE_DIR = r"E:\TreeExtraction\Data_v1\patches\images"  
LABEL_DIR = r"E:\TreeExtraction\Data_v1\patches\labels"
YOLO_DIR = r"E:\TreeExtraction\Data_v1\yolo_data"

IMG_DIR = os.path.join(YOLO_DIR, "images")
LBL_DIR = os.path.join(YOLO_DIR, "labels")
os.makedirs(IMG_DIR, exist_ok=True)
os.makedirs(LBL_DIR, exist_ok=True)

patch_files = [f for f in os.listdir(IMAGE_DIR) if f.endswith(".tif")]
print(f"🔎 Found {len(patch_files)} patch images")

count_with_labels = 0

for patch_name in patch_files:
    patch_path = os.path.join(IMAGE_DIR, patch_name)
    lbl_file = patch_name.replace(".tif", ".txt")
    lbl_path = os.path.join(LABEL_DIR, lbl_file)
    
    shutil.copy(patch_path, os.path.join(IMG_DIR, patch_name))

    if os.path.exists(lbl_path):
        shutil.copy(lbl_path, os.path.join(LBL_DIR, lbl_file))
        count_with_labels += 1

print(f"✅ Copied {count_with_labels} images with labels to YOLO folder")

all_images = [f for f in os.listdir(IMG_DIR) if f.endswith(".tif")]
random.shuffle(all_images)

train_split = int(0.7 * len(all_images))
val_split = int(0.9 * len(all_images))

splits = {
    "train": all_images[:train_split],
    "val": all_images[train_split:val_split],
    "test": all_images[val_split:],
}

for split, files in splits.items():
    split_img_dir = os.path.join(YOLO_DIR, split, "images")
    split_lbl_dir = os.path.join(YOLO_DIR, split, "labels")
    os.makedirs(split_img_dir, exist_ok=True)
    os.makedirs(split_lbl_dir, exist_ok=True)

    for f in files:
        shutil.move(os.path.join(IMG_DIR, f), os.path.join(split_img_dir, f))
        lbl_file = f.replace(".tif", ".txt")
        if os.path.exists(os.path.join(LBL_DIR, lbl_file)):
            shutil.move(os.path.join(LBL_DIR, lbl_file), os.path.join(split_lbl_dir, lbl_file))

print("✅ Dataset split into Train/Val/Test")

yaml_content = f"""
train: {os.path.join(YOLO_DIR, 'train')}
val: {os.path.join(YOLO_DIR, 'val')}
test: {os.path.join(YOLO_DIR, 'test')}

nc: 1
names: ['coconut_tree']
"""

with open(os.path.join(YOLO_DIR, "data.yaml"), "w") as f:
    f.write(yaml_content)

print("✅ data.yaml file created")


🔎 Found 772 patch images
✅ Copied 772 images with labels to YOLO folder
✅ Dataset split into Train/Val/Test
✅ data.yaml file created


In [1]:
import os
from ultralytics import YOLO

DATA_DIR = r"E:\TreeExtraction\Data_v1\yolo_data"   
DATA_YAML = os.path.join(DATA_DIR, "data.yaml")     
SAVE_DIR = os.path.join(DATA_DIR, "runs")          
os.makedirs(SAVE_DIR, exist_ok=True)

model = YOLO("yolo11n.yaml")

# === Train model ===
model.train(
    data=DATA_YAML,
    epochs=2000,
    imgsz=256,
    batch=32,             
    device=0,
    workers=8,
    optimizer="AdamW",
    patience=0,
    lr0=0.0001,
    pretrained=False,
    save=True,
    save_period=10,
    val=True,
    project=SAVE_DIR,
    name="yolov11_scratch"
)

metrics = model.val(data=DATA_YAML, imgsz=256, device=0)
print("Final Evaluation Metrics:", metrics)


New https://pypi.org/project/ultralytics/8.3.186 available  Update with 'pip install -U ultralytics'
Ultralytics 8.3.185  Python-3.10.16 torch-2.7.1+cu118 CUDA:0 (NVIDIA GeForce GTX 1650, 4096MiB)
engine\trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=32, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=E:\TreeExtraction\Data_v1\yolo_data\data.yaml, degrees=0.0, deterministic=True, device=0, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=2000, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=256, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.0001, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=yolo11n.yaml, momentum=0.937, mosaic=1.0, multi_scale=False, name=yolov11_scratch, nb

train: Scanning E:\TreeExtraction\Data_v1\yolo_data\train\labels.cache... 540 images, 0 backgrounds, 0 corrupt: 100%|██


albumentations: Blur(p=0.01, blur_limit=(3, 7)), MedianBlur(p=0.01, blur_limit=(3, 7)), ToGray(p=0.01, num_output_channels=3, method='weighted_average'), CLAHE(p=0.01, clip_limit=(1.0, 4.0), tile_grid_size=(8, 8))
val: Fast image access  (ping: 0.10.1 ms, read: 514.5191.3 MB/s, size: 194.0 KB)


val: Scanning E:\TreeExtraction\Data_v1\yolo_data\val\labels.cache... 154 images, 0 backgrounds, 0 corrupt: 100%|██████


Plotting labels to E:\TreeExtraction\Data_v1\yolo_data\runs\yolov11_scratch\labels.jpg... 
optimizer: AdamW(lr=0.0001, momentum=0.937) with parameter groups 81 weight(decay=0.0), 88 weight(decay=0.0005), 87 bias(decay=0.0)
Image sizes 256 train, 256 val
Using 8 dataloader workers
Logging results to E:\TreeExtraction\Data_v1\yolo_data\runs\yolov11_scratch
Starting training for 2000 epochs...

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


...


2000 epochs completed in 5.619 hours.
Optimizer stripped from E:\TreeExtraction\Data_v1\yolo_data\runs\yolov11_scratch\weights\last.pt, 5.7MB
Optimizer stripped from E:\TreeExtraction\Data_v1\yolo_data\runs\yolov11_scratch\weights\best.pt, 5.7MB

Validating E:\TreeExtraction\Data_v1\yolo_data\runs\yolov11_scratch\weights\best.pt...
Ultralytics 8.3.185  Python-3.10.16 torch-2.7.1+cu118 CUDA:0 (NVIDIA GeForce GTX 1650, 4096MiB)
YOLO11n summary (fused): 100 layers, 2,582,347 parameters, 0 gradients, 6.3 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 3/3 [00:02<0


                   all        154        757      0.886      0.802       0.85      0.507
Speed: 0.4ms preprocess, 5.1ms inference, 0.0ms loss, 2.5ms postprocess per image
Results saved to E:\TreeExtraction\Data_v1\yolo_data\runs\yolov11_scratch
Ultralytics 8.3.185  Python-3.10.16 torch-2.7.1+cu118 CUDA:0 (NVIDIA GeForce GTX 1650, 4096MiB)
YOLO11n summary (fused): 100 layers, 2,582,347 parameters, 0 gradients, 6.3 GFLOPs
val: Fast image access  (ping: 0.10.0 ms, read: 936.4106.7 MB/s, size: 194.0 KB)


val: Scanning E:\TreeExtraction\Data_v1\yolo_data\val\labels.cache... 154 images, 0 backgrounds, 0 corrupt: 100%|██████
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 5/5 [00:11<0


                   all        154        757      0.886      0.802       0.85      0.507
Speed: 0.6ms preprocess, 8.1ms inference, 0.0ms loss, 2.6ms postprocess per image
Results saved to E:\TreeExtraction\Data_v1\yolo_data\runs\yolov11_scratch2
Final Evaluation Metrics: ultralytics.utils.metrics.DetMetrics object with attributes:

ap_class_index: array([0])
box: ultralytics.utils.metrics.Metric object
confusion_matrix: <ultralytics.utils.metrics.ConfusionMatrix object at 0x000001595B7E5540>
curves: ['Precision-Recall(B)', 'F1-Confidence(B)', 'Precision-Confidence(B)', 'Recall-Confidence(B)']
curves_results: [[array([          0,    0.001001,    0.002002,    0.003003,    0.004004,    0.005005,    0.006006,    0.007007,    0.008008,    0.009009,     0.01001,    0.011011,    0.012012,    0.013013,    0.014014,    0.015015,    0.016016,    0.017017,    0.018018,    0.019019,     0.02002,    0.021021,    0.022022,    0.023023,
          0.024024,    0.025025,    0.026026,    0.027027,    0

In [13]:
import os
from ultralytics import YOLO

DATA_DIR = r"E:\TreeExtraction\Data_v1\yolo_data"
DATA_YAML = os.path.join(DATA_DIR, "data.yaml")
WEIGHTS = os.path.join(DATA_DIR, "runs", "yolov11_scratch", "weights", "best.pt") 
SAVE_DIR = os.path.join(DATA_DIR, "runs", "yolov11_scratch_inference")
os.makedirs(SAVE_DIR, exist_ok=True)

model = YOLO(WEIGHTS)

results = model.predict(
    source=os.path.join(DATA_DIR, "test", "images"),  
    imgsz=256,
    conf=0.5,    
    iou=0.5,      
    save=True,      
    save_txt=False, 
    project=SAVE_DIR,
    name="test_predictions",
    exist_ok=True
)


print("✅ Predictions saved. Compare predicted vs ground truth visually.")



image 1/78 E:\TreeExtraction\Data_v1\yolo_data\test\images\image_002.tif: 256x256 3 coconut_trees, 24.0ms
image 2/78 E:\TreeExtraction\Data_v1\yolo_data\test\images\image_033.tif: 256x256 6 coconut_trees, 30.6ms
image 3/78 E:\TreeExtraction\Data_v1\yolo_data\test\images\image_043.tif: 256x256 6 coconut_trees, 24.3ms
image 4/78 E:\TreeExtraction\Data_v1\yolo_data\test\images\image_047.tif: 256x256 1 coconut_tree, 23.9ms
image 5/78 E:\TreeExtraction\Data_v1\yolo_data\test\images\image_073.tif: 256x256 1 coconut_tree, 40.8ms
image 6/78 E:\TreeExtraction\Data_v1\yolo_data\test\images\image_081.tif: 256x256 4 coconut_trees, 24.2ms
image 7/78 E:\TreeExtraction\Data_v1\yolo_data\test\images\image_100.tif: 256x256 5 coconut_trees, 24.4ms
image 8/78 E:\TreeExtraction\Data_v1\yolo_data\test\images\image_108.tif: 256x256 1 coconut_tree, 33.0ms
image 9/78 E:\TreeExtraction\Data_v1\yolo_data\test\images\image_113.tif: 256x256 2 coconut_trees, 27.8ms
image 10/78 E:\TreeExtraction\Data_v1\yolo_data\

In [5]:
import os
from ultralytics import YOLO
import geopandas as gpd
from shapely.geometry import box
import rasterio

DATA_DIR = r"E:\TreeExtraction\Data_v1\yolo_data"
DATA_YAML = os.path.join(DATA_DIR, "data.yaml")
WEIGHTS = os.path.join(DATA_DIR, "runs", "yolov11_scratch", "weights", "best.pt") 
SAVE_DIR = os.path.join(DATA_DIR, "runs", "yolov11_scratch_inference")
os.makedirs(SAVE_DIR, exist_ok=True)

IMAGE_PATH = r"E:\TreeExtraction\Data_v1\Test_Image_.tif"  

model = YOLO(WEIGHTS)

results = model.predict(
    source=IMAGE_PATH,
    imgsz=2048,
    conf=0.05,
    iou=0.3,
    save=True,
    project=SAVE_DIR,
    name="full_image_predictions",
    exist_ok=True,
    show_labels=True,   
    show_conf=False     
)

print("✅ Inference completed, image saved with bounding boxes.")

with rasterio.open(IMAGE_PATH) as src:
    transform = src.transform
    crs = src.crs

def pixel_to_coords(x, y, transform):
    """Convert pixel to geographic coordinates"""
    x_geo, y_geo = rasterio.transform.xy(transform, y, x)
    return x_geo, y_geo

all_geoms = []
all_classes = []

for r in results:
    boxes = r.boxes.xyxy.cpu().numpy()   # x1, y1, x2, y2
    clss = r.boxes.cls.cpu().numpy()     # class IDs
    names = r.names                      # class names dictionary
    
    for box_xyxy, cls_id in zip(boxes, clss):
        x1, y1, x2, y2 = box_xyxy

        # convert pixel → geo coords
        x1g, y1g = pixel_to_coords(x1, y1, transform)
        x2g, y2g = pixel_to_coords(x2, y2, transform)

        geom = box(x1g, y2g, x2g, y1g)
        all_geoms.append(geom)
        all_classes.append(names[int(cls_id)])

gdf = gpd.GeoDataFrame({"class": all_classes, "geometry": all_geoms}, crs=crs)

SHP_PATH = os.path.join(SAVE_DIR, "Test_Image_predicted_boxes.shp")
gdf.to_file(SHP_PATH)

print(f"✅ Shapefile saved at: {SHP_PATH}")


image 1/1 E:\TreeExtraction\Data_v1\Test_Image_.tif: 1184x2048 109 coconut_trees, 69.2ms
Speed: 70.9ms preprocess, 69.2ms inference, 49.4ms postprocess per image at shape (1, 3, 1184, 2048)
Results saved to E:\TreeExtraction\Data_v1\yolo_data\runs\yolov11_scratch_inference\full_image_predictions
✅ Inference completed, image saved with bounding boxes.
✅ Shapefile saved at: E:\TreeExtraction\Data_v1\yolo_data\runs\yolov11_scratch_inference\Test_Image_predicted_boxes.shp
